In [1]:
# Cell 1: scripts folder to path & import modules for whole coding
import sys
from pathlib import Path
import time
from datetime import timedelta
# 1) Locate project root (one level up from this notebooks directory)
try:
    project_root = Path(__file__).resolve().parent.parent
except NameError:
    project_root = Path.cwd().parent
    
print(f"Project root identified as: {project_root}")
# 2) Point at your scripts folder
scripts_dir = project_root / "5_nsga" / "5_nsga_scripts"
assert scripts_dir.exists(), f"Can't find scripts at {scripts_dir}"
sys.path.insert(0, str(scripts_dir))
# 3) Import helper modules
from pv_simulate_30 import simulate_multi_year_pv
from battery_30 import simulate_battery_dispatch
from financial_30 import compute_financials
from objective_30 import evaluate_solution
import objective_30
# 4) Import optimiser & data libs
import numpy as np
import pandas as pd
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.termination import get_termination
from pymoo.optimize import minimize
from pymoo.core.callback import Callback
print("All modules imported successfully - can move onto next stage")

Project root identified as: /Users/petertunali/Documents/GitHub/Battery_Optimisation
All modules imported successfully - can move onto next stage


In [2]:
# Cell 2: Configure outputs, check data & build 30‑yr demand
from pathlib import Path
import pandas as pd
# 1) Auto‑version your outputs folder under project_root/outputs_optimisation
base_out = project_root / "5_nsga_outputs"
base_out.mkdir(exist_ok=True)
existing = [d.name for d in base_out.iterdir() if d.is_dir() and d.name.isdigit()]
nums     = sorted(int(n) for n in existing) if existing else []
next_run = nums[-1] + 1 if nums else 1
out_dir  = base_out / f"{next_run:03d}"
out_dir.mkdir()
print(f"Writing all outputs to → {out_dir}")
# 2) Locate your data directory
data_dir = project_root / "data"
if not data_dir.exists():
    for alt in [project_root/"Battery_Optimisation"/"data", Path.cwd()/"data"]:
        if alt.exists():
            data_dir = alt
            break
print(f"Using data directory → {data_dir}")
# 3) Check EPW weather files
weather_files = [
    str(data_dir / "Bonfire_2025.epw"),
    str(data_dir / "Bonfire_2040_4_5.epw"),
    str(data_dir / "Bonfire_2050_4_5.epw")
]
print("\nChecking weather files:")
for wf in weather_files:
    print(" ✔ Found:" if Path(wf).exists() else " ❌ Missing:", wf)
assert all(Path(wf).exists() for wf in weather_files), "One or more weather files missing!"
# 4) Find demand CSV (take the first match)
demand_paths = [
    data_dir / "PV_Generation_excel.csv",
    data_dir / "Energy_Demand_and_Supply_2024.csv"
]
for p in demand_paths:
    if p.exists():
        demand_file = p
        print("\n✔ Found demand file:", p)
        break
else:
    raise FileNotFoundError("No demand file found – check your filenames")

# 5) Simplified demand load function
def load_demand_profile(csv_path: Path) -> pd.Series:
    # Load the CSV and skip NaN timestamp rows immediately
    raw = pd.read_csv(csv_path, parse_dates=['Date and Time'], dayfirst=True)
    
    # Drop rows with NaN timestamps right away
    raw = raw.dropna(subset=['Date and Time'])
    print(f"CSV loaded with {len(raw)} valid rows")
    
    # Get the consumption column
    consumption_col = 'Consumtpion (kWh)'  # Based on your data sample
    if consumption_col not in raw.columns:
        # Try to find the consumption column
        for col in raw.columns:
            if 'consum' in col.lower() or 'demand' in col.lower():
                consumption_col = col
                break
        else:
            # If no consumption column found, use the second column
            consumption_col = raw.columns[1]
    
    print(f"Using consumption column: '{consumption_col}'")
    
    # Create a Series with timestamp index and consumption values
    s = pd.Series(raw[consumption_col].values, index=raw['Date and Time'])
    
    # Check for duplicate timestamps
    dup_count = s.index.duplicated().sum()
    if dup_count:
        print(f"⚠️ Dropping {dup_count} duplicate timestamps")
        s = s[~s.index.duplicated(keep='first')]
    
    # Build the expected half-hour index for entire year (no Feb 29)
    year = s.index.min().year
    start = pd.Timestamp(year, 1, 1, 0, 0)
    end = pd.Timestamp(year, 12, 31, 23, 30)
    expected = pd.date_range(start, end, freq="30min")
    expected = expected[~((expected.month==2) & (expected.day==29))]
    
    # Reindex to ensure complete coverage
    s = s.reindex(expected)
    missing = s.isna().sum()
    if missing:
        print(f"⚠️ Filling {missing} missing points with 0")
        s = s.fillna(0.0)
    
    # Final sanity check
    assert len(s) == 17520, f"Got {len(s)} points, expected 17520"
    return s

# 6) Load one-year demand data and build 30-year profile
print(f"Loading demand data from: {demand_file}\n")
one_year_dem = load_demand_profile(demand_file)
print(f"One year data points: {len(one_year_dem)}")

# 7) Create 30-year demand profile with proper date handling
start_year = one_year_dem.index[0].year
years = 30
all_data = []

# Build 30 years of data year by year
for year_offset in range(years):
    # Copy the data for this year
    year_data = one_year_dem.copy()
    
    # Create index for this specific year
    year_start = pd.Timestamp(start_year + year_offset, 1, 1, 0, 0)
    year_end = pd.Timestamp(start_year + year_offset, 12, 31, 23, 30)
    year_range = pd.date_range(start=year_start, end=year_end, freq="30min")
    
    # Remove Feb 29 if it's a leap year
    year_range = year_range[~((year_range.month == 2) & (year_range.day == 29))]
    
    # Make sure it has the right number of points
    assert len(year_range) == len(one_year_dem), f"Year {start_year + year_offset} has {len(year_range)} points, expected {len(one_year_dem)}"
    
    # Assign the new index and add to our list
    year_data.index = year_range
    all_data.append(year_data)

# Concatenate all years
demand_profile = pd.concat(all_data)
print("\n30‑year demand profile built:")
print(f"  • Time steps : {len(demand_profile)}")
print(f"  • Date range : {demand_profile.index[0]} → {demand_profile.index[-1]}")
print(f"  • Total demand: {demand_profile.sum():.2f} kWh")

Writing all outputs to → /Users/petertunali/Documents/GitHub/Battery_Optimisation/5_nsga_outputs/008
Using data directory → /Users/petertunali/Documents/GitHub/Battery_Optimisation/data

Checking weather files:
 ✔ Found: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/Bonfire_2025.epw
 ✔ Found: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/Bonfire_2040_4_5.epw
 ✔ Found: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/Bonfire_2050_4_5.epw

✔ Found demand file: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/PV_Generation_excel.csv
Loading demand data from: /Users/petertunali/Documents/GitHub/Battery_Optimisation/data/PV_Generation_excel.csv

CSV loaded with 17520 valid rows
Using consumption column: 'Consumtpion (kWh)'
One year data points: 17520

30‑year demand profile built:
  • Time steps : 525600
  • Date range : 2025-01-01 00:00:00 → 2054-12-31 23:30:00
  • Total demand: 1318609.49 kWh


In [3]:
# Cell 3: Define the PV systems and roof parameters
# Define PV system parameters - keep your original 10kW configuration
roof_params = [{
    'name': 'main_roof',
    'system_capacity_kw': 10.0,  # 10kW system
    'tilt': 10.0,
    'azimuth': 18.0,  # North-facing in Southern Hemisphere
    'shading': 43.0,  # 43% shading impact
}]
print("Roof parameters:")
for roof in roof_params:
    print(f"  - {roof['name']}: {roof['system_capacity_kw']} kW, Tilt: {roof['tilt']}°, Azimuth: {roof['azimuth']}°, Shading: {roof.get('shading', 0.0)}%")
total_pv_capacity = sum(roof['system_capacity_kw'] for roof in roof_params)
print(f"Total PV capacity: {total_pv_capacity} kW")
print("ready for next stage")

Roof parameters:
  - main_roof: 10.0 kW, Tilt: 10.0°, Azimuth: 18.0°, Shading: 43.0%
Total PV capacity: 10.0 kW
ready for next stage


In [4]:
# Cell 4: Build 30‑yr PV & demand, then compute zero‑battery baseline
print("1) Simulating 30 yr PV…")
start_time = time.time()
start_years = [2025, 2040, 2050]
pv_profile = simulate_multi_year_pv(
    weather_files    = weather_files,
    roof_params      = roof_params,
    repeats_per_file = 10,
    start_years      = start_years
)
elapsed = time.time() - start_time
print(f"PV simulation completed in {elapsed:.1f} seconds ({timedelta(seconds=int(elapsed))})")
print(f"   • PV steps : {len(pv_profile)}")
print(f"   • Date range: {pv_profile.index[0]} → {pv_profile.index[-1]}")
print(f"   • Gen total : {pv_profile['simulated_kwh'].sum():.2f} kWh\n")

# 4b) Compute baselines for no-PV and PV-only scenarios
print("Computing baseline costs...")
start_time = time.time()

# Manually set annual costs from your electricity bill data
ANNUAL_NO_PV_COST = 9424.48      # From your bill - no PV, no battery
ANNUAL_PV_ONLY_COST = 8246.44    # From your bill - with PV, no battery

# Calculate 30-year baseline costs (simple multiplication)
NO_PV_BASELINE_COST = ANNUAL_NO_PV_COST * 30
PV_ONLY_BASELINE_COST = ANNUAL_PV_ONLY_COST * 30

print(f"Annual no-PV cost: ${ANNUAL_NO_PV_COST:,.2f}")
print(f"Annual PV-only cost: ${ANNUAL_PV_ONLY_COST:,.2f}")
print(f"30-year no-PV baseline: ${NO_PV_BASELINE_COST:,.2f}")
print(f"30-year PV-only baseline: ${PV_ONLY_BASELINE_COST:,.2f}")
print(f"Annual savings from PV: ${ANNUAL_NO_PV_COST - ANNUAL_PV_ONLY_COST:,.2f}")

# Optional - Calculate PV-only baseline through simulation
disp0, totals0 = simulate_battery_dispatch(
    pv_gen      = pv_profile['simulated_kwh'],
    demand      = demand_profile,
    battery_kwh = 0.0,
    roundtrip_eff = 0.9,
    min_soc_pct = 0.05,
    annual_deg_rate = 0.01,
    grid_emission_rate = 0.81
)

# Calculate PV-only annual cost from simulation
simulated_pv_only_import_cost = (
    totals0['total_grid_import_peak'] * 0.39710 + 
    totals0['total_grid_import_offpeak'] * 0.13530
)
simulated_pv_only_export_rev = totals0['total_pv_export'] * 0.033
SIMULATED_PV_ONLY_ANNUAL_COST = (simulated_pv_only_import_cost - simulated_pv_only_export_rev) / 30
SIMULATED_PV_ONLY_TOTAL_COST = simulated_pv_only_import_cost - simulated_pv_only_export_rev

print(f"Simulated annual PV-only cost: ${SIMULATED_PV_ONLY_ANNUAL_COST:,.2f}")
print(f"Simulated 30-year PV-only cost: ${SIMULATED_PV_ONLY_TOTAL_COST:,.2f}")

# Use your actual bill values for the baselines
BASE_IMPORT_COST = NO_PV_BASELINE_COST       # No PV, no battery
PV_ONLY_COST = PV_ONLY_BASELINE_COST         # PV-only, no battery

# Set these values in the objective function
objective_30.BASE_IMPORT_COST = BASE_IMPORT_COST
objective_30.PV_ONLY_COST = PV_ONLY_COST

print(f"   → Baseline no-PV cost over 30 yr = ${BASE_IMPORT_COST:,.2f}")
print(f"   → Baseline PV-only cost over 30 yr = ${PV_ONLY_COST:,.2f}")

elapsed = time.time() - start_time
print(f"Calculation completed in {elapsed:.1f} seconds ({timedelta(seconds=int(elapsed))})")

1) Simulating 30 yr PV…
PV simulation completed in 0.2 seconds (0:00:00)
   • PV steps : 525600
   • Date range: 2025-01-01 00:00:00 → 2054-12-24 23:30:00
   • Gen total : 204028.14 kWh

Computing baseline costs...
Annual no-PV cost: $9,424.48
Annual PV-only cost: $8,246.44
30-year no-PV baseline: $282,734.40
30-year PV-only baseline: $247,393.20
Annual savings from PV: $1,178.04
Simulated annual PV-only cost: $8,249.58
Simulated 30-year PV-only cost: $247,487.34
   → Baseline no-PV cost over 30 yr = $282,734.40
   → Baseline PV-only cost over 30 yr = $247,393.20
Calculation completed in 1.2 seconds (0:00:01)


In [5]:
# Debug Cell: Detailed battery economics analysis
import time
import sys
import numpy as np
from datetime import timedelta
import pandas as pd
from numpy_financial import npv, irr  # Import both irr and npv from numpy_financial

print("Analyzing battery economics in detail...")

# Define actual electricity costs from your bill
ANNUAL_NO_PV_COST = 9424.48      # From your bill - no PV, no battery
ANNUAL_PV_ONLY_COST = 8246.44    # From your bill - with PV, no battery

# Calculate 30-year baseline costs
NO_PV_BASELINE_COST = ANNUAL_NO_PV_COST * 30
PV_ONLY_BASELINE_COST = ANNUAL_PV_ONLY_COST * 30

# Define maintenance costs
battery_maintenance_per_kw_day = 0.12  # $0.12 per kW per day for battery
pv_maintenance_per_kw_day = 0.13  # $0.13 per kW per day for PV

print(f"Annual no-PV cost: ${ANNUAL_NO_PV_COST:,.2f}")
print(f"Annual PV-only cost: ${ANNUAL_PV_ONLY_COST:,.2f}")
print(f"Annual savings from PV alone: ${ANNUAL_NO_PV_COST - ANNUAL_PV_ONLY_COST:,.2f}")

# Test with a range of battery sizes
battery_sizes = [0.0, 5.0, 10.0, 15.0, 20.0, 30.0, 40.0, 50.0]
results = []

# Check if key variables exist, if not create them
if 'pv_profile' not in globals() or 'demand_profile' not in globals() or 'total_pv_capacity' not in globals():
    print("Required variables not found. Make sure to run Cells 3 and 4 first.")
    # Define defaults just to allow the cell to run
    if 'total_pv_capacity' not in globals():
        total_pv_capacity = 10.0
        print(f"Using default PV capacity: {total_pv_capacity} kW")

for battery_kwh in battery_sizes:
    print(f"\nAnalyzing {battery_kwh} kWh battery...")
    # Simulate with this battery size
    disp, totals = simulate_battery_dispatch(
        pv_gen=pv_profile['simulated_kwh'],
        demand=demand_profile,
        battery_kwh=battery_kwh,
        battery_kw=battery_kwh * 0.5,  # 0.5C power rating
        roundtrip_eff=0.9,             # 90% round trip efficiency
        min_soc_pct=0.05,              # 95% depth of discharge (5% min SOC)
        annual_deg_rate=0.01,          # 1% annual degradation fallback rate
        grid_emission_rate=0.81        # 0.81 kg CO2e/kWh
    )
    
    # Calculate battery power in kW (0.5C rate)
    battery_kw = battery_kwh * 0.5
    
    # Calculate battery cost using formula with min $600/kWh
    if battery_kwh <= 0:
        battery_cost_per_kwh = 0.0  # No cost for 0 kWh battery
    else:
        battery_cost_per_kwh = 977.54 * np.exp(-0.004 * battery_kwh)
        battery_cost_per_kwh = max(600.0, battery_cost_per_kwh)  # Minimum $600/kWh
    
    print(f"  Battery cost: ${battery_cost_per_kwh:.2f} per kWh")
    
    # Calculate annual costs from simulation
    annual_import_cost = (
        totals['total_grid_import_peak'] * 0.39710 + 
        totals['total_grid_import_offpeak'] * 0.13530
    ) / 30
    annual_export_revenue = totals['total_pv_export'] * 0.033 / 30
    annual_bill = annual_import_cost - annual_export_revenue
    
    # Calculate annual maintenance cost
    annual_batt_maintenance = battery_maintenance_per_kw_day * battery_kw * 365 if battery_kw > 0 else 0
    
    # Calculate savings compared to PV-only scenario
    annual_savings_vs_pv_only = ANNUAL_PV_ONLY_COST - annual_bill
    
    # Calculate simple payback period
    if battery_kwh > 0 and annual_savings_vs_pv_only > 0:
        battery_capex = battery_kwh * (battery_cost_per_kwh + 174.0)  # Including installation
        simple_payback = battery_capex / annual_savings_vs_pv_only
    else:
        simple_payback = float('inf')
    
    # Calculate annual cash flow and IRR
    if battery_kwh > 0:
        annual_cash_flows = [annual_savings_vs_pv_only - annual_batt_maintenance] * 30
        cash_flows = [-battery_kwh * (battery_cost_per_kwh + 174.0)] + annual_cash_flows
        try:
            irr_value = irr(cash_flows)
        except:
            irr_value = None
    else:
        irr_value = None
        cash_flows = [0]
    
    # Calculate NPV
    if battery_kwh > 0:
        try:
            npv_value = npv(0.07, cash_flows)
        except:
            npv_value = None
    else:
        npv_value = 0
    
    # Store all metrics for this battery size
    results.append({
        'battery_kwh': battery_kwh,
        'battery_cost_per_kwh': battery_cost_per_kwh,
        'capex': battery_kwh * (battery_cost_per_kwh + 174.0) if battery_kwh > 0 else 0,
        'annual_maintenance': annual_batt_maintenance,
        'annual_bill': annual_bill,
        'annual_import_cost': annual_import_cost,
        'annual_export_revenue': annual_export_revenue,
        'annual_savings_vs_pv_only': annual_savings_vs_pv_only,
        'simple_payback_years': simple_payback,
        'irr': irr_value,
        'npv': npv_value,
        'battery_cycles': totals.get('battery_cycles', 0),
        'final_degradation_pct': totals.get('final_degradation_pct', 0),
        'renewable_fraction': totals.get('renewable_fraction', 0)
    })

# Convert to DataFrame for display
debug_df = pd.DataFrame(results)

# Display key economic metrics
print("\nEconomic Results for different battery sizes (assuming existing PV):")
econ_cols = ['battery_kwh', 'battery_cost_per_kwh', 'capex', 'annual_maintenance', 
             'annual_bill', 'annual_savings_vs_pv_only', 'simple_payback_years', 'irr', 'npv']
print(debug_df[econ_cols].to_string(index=False))

# Display technical metrics
print("\nTechnical Performance:")
tech_cols = ['battery_kwh', 'battery_cycles', 'final_degradation_pct', 'renewable_fraction']
print(debug_df[tech_cols].to_string(index=False))

# Print annual bill comparison
print("\nAnnual Bill Comparison:")
print(f"No PV, no battery: ${ANNUAL_NO_PV_COST:,.2f}/year")
print(f"PV only, no battery: ${ANNUAL_PV_ONLY_COST:,.2f}/year")

for idx, row in debug_df.iterrows():
    if row['battery_kwh'] > 0:
        print(f"\nWith {row['battery_kwh']} kWh battery:")
        print(f"  Annual electricity bill: ${row['annual_bill']:,.2f}/year")
        print(f"  Annual savings vs. PV-only: ${row['annual_savings_vs_pv_only']:,.2f}/year")
        print(f"  Battery cost: ${row['capex']:,.2f}")
        print(f"  Simple payback: {row['simple_payback_years']:.1f} years")
        if not pd.isna(row['irr']):
            print(f"  IRR: {row['irr']*100:.2f}%")
        else:
            print(f"  IRR: N/A")
        break  # Just show the first non-zero battery size

# Verify if simulation matches your bill data
print("\nVerification of simulation vs. your electricity bill data:")
if 0 in debug_df['battery_kwh'].values:
    zero_batt_idx = debug_df[debug_df['battery_kwh'] == 0].index[0]
    print(f"Annual bill with 0 kWh battery (simulated PV-only): ${debug_df.loc[zero_batt_idx, 'annual_bill']:,.2f}/year")
    print(f"Annual bill from your data (actual PV-only): ${ANNUAL_PV_ONLY_COST:,.2f}/year")
    print(f"Difference: ${debug_df.loc[zero_batt_idx, 'annual_bill'] - ANNUAL_PV_ONLY_COST:,.2f}/year")
    
    # Calculate percentage difference
    pct_diff = abs(debug_df.loc[zero_batt_idx, 'annual_bill'] - ANNUAL_PV_ONLY_COST) / ANNUAL_PV_ONLY_COST * 100
    print(f"Percentage difference: {pct_diff:.2f}%")
    
    if pct_diff > 10:
        print("\nWARNING: The simulation results differ significantly from your actual bill.")
        print("This may indicate that the PV generation or demand profiles in the model")
        print("don't match your actual usage patterns. You might want to calibrate the model.")
    else:
        print("\nThe simulation results are reasonably close to your actual bill data.")

# Find if any battery size is economically viable
viable_batteries = debug_df[(~pd.isna(debug_df['irr'])) & (debug_df['irr'] > 0.07)]  # 7% threshold
if not viable_batteries.empty:
    best_idx = viable_batteries['irr'].idxmax()
    best_size = viable_batteries.loc[best_idx, 'battery_kwh']
    best_irr = viable_batteries.loc[best_idx, 'irr']
    best_npv = viable_batteries.loc[best_idx, 'npv']
    best_payback = viable_batteries.loc[best_idx, 'simple_payback_years']
    
    print("\nBest economically viable battery size:")
    print(f"Battery size: {best_size:.1f} kWh")
    print(f"IRR: {best_irr*100:.2f}%")
    print(f"NPV: ${best_npv:,.2f}")
    print(f"Simple payback: {best_payback:.1f} years")
    print(f"Final degradation after 30 years: {viable_batteries.loc[best_idx, 'final_degradation_pct']:.1f}%")
    print(f"Equivalent full cycles: {viable_batteries.loc[best_idx, 'battery_cycles']:,.1f}")
else:
    print("\nNo battery size meets the minimum 7% IRR threshold for economic viability.")
    print("Consider exploring other scenarios with different electricity rates or battery costs.")
    
    # Find the best option even if not viable
    non_na_irr = debug_df[~pd.isna(debug_df['irr'])]
    if not non_na_irr.empty:
        least_negative_idx = non_na_irr['irr'].idxmax()
        print(f"\nLeast negative IRR: {non_na_irr.loc[least_negative_idx, 'irr']*100:.2f}% with {non_na_irr.loc[least_negative_idx, 'battery_kwh']:.1f} kWh battery")
        print(f"Payback period: {non_na_irr.loc[least_negative_idx, 'simple_payback_years']:.1f} years")

Analyzing battery economics in detail...
Annual no-PV cost: $9,424.48
Annual PV-only cost: $8,246.44
Annual savings from PV alone: $1,178.04

Analyzing 0.0 kWh battery...
  Battery cost: $0.00 per kWh

Analyzing 5.0 kWh battery...
  Battery cost: $958.18 per kWh

Analyzing 10.0 kWh battery...
  Battery cost: $939.21 per kWh

Analyzing 15.0 kWh battery...
  Battery cost: $920.61 per kWh

Analyzing 20.0 kWh battery...
  Battery cost: $902.38 per kWh

Analyzing 30.0 kWh battery...
  Battery cost: $867.00 per kWh

Analyzing 40.0 kWh battery...
  Battery cost: $833.00 per kWh

Analyzing 50.0 kWh battery...
  Battery cost: $800.34 per kWh

Economic Results for different battery sizes (assuming existing PV):
 battery_kwh  battery_cost_per_kwh        capex  annual_maintenance  annual_bill  annual_savings_vs_pv_only  simple_payback_years       irr           npv
         0.0              0.000000     0.000000                 0.0  8249.578086                  -3.138086                   inf      

In [6]:
# Cell 6: Define the optimisation problem to solve
print("Defining optimization problem...")
try:
    # Define the NSGA-II optimization problem
    class BatteryOptimizationProblem(Problem):
        def __init__(self):
            # Define one variable (battery size in kWh)
            # with lower bound 0 and upper bound 50
            super().__init__(
                n_var=1,             # Number of decision variables
                n_obj=2,             # Number of objectives
                n_ieq_constr=0,      # Number of inequality constraints
                xl=np.array([0.0]),  # Lower bounds of variables
                xu=np.array([50.0])  # Upper bounds of variables
            )
            # Store the profiles for use in evaluation
            self.pv_profile = pv_profile
            self.demand_profile = demand_profile
        
        def _evaluate(self, x, out, *args, **kwargs):
            # Evaluate each solution in the population
            n_solutions = x.shape[0]
            F = np.zeros((n_solutions, 2))  # For storing [-IRR, NPC] values
            
            # Loop through each solution (battery size)
            for i in range(n_solutions):
                battery_kwh = x[i, 0]
                # Define parameters for evaluation
                params = {
                    'battery_kwh': battery_kwh,
                    'pv_kw': total_pv_capacity
                }
                # Evaluate the solution using your objective function
                # returns [-IRR, NPC]
                F[i, :] = evaluate_solution(params, self.pv_profile, self.demand_profile)
            
            # Set the output objectives array
            out["F"] = F
    
    # Create an instance of the problem
    problem = BatteryOptimizationProblem()
    print("Optimization problem defined successfully")
    
except Exception as e:
    print(f"Error defining optimization problem: {e}")
    raise

Defining optimization problem...
Optimization problem defined successfully


In [7]:
# Cell 7: Define callback for live tracking during optimization
class BestSolutionCallback(Callback):
    def __init__(self):
        super().__init__()
        self.start_time = time.time()
        self.evaluations = 0
        self.data = {
            'gen': [],
            'best_irr': [],
            'best_npv': [],  # Changed from NPC to NPV
            'batt_irr': [],
            'batt_npv': [],  # Changed from NPC to NPV
            'time_elapsed': []
        }
    
    def notify(self, algorithm):
        elapsed = time.time() - self.start_time
        F = algorithm.pop.get("F")      # shape (pop_size, 2): [–IRR, -NPV]
        X = algorithm.pop.get("X")[:,0] # battery sizes
        
        # Count evaluations (pop_size per generation)
        self.evaluations += len(X)
        
        # best IRR solution
        idx_irr = np.argmin(F[:,0])
        best_irr = -F[idx_irr,0]
        batt_irr = X[idx_irr]
        
        # best NPV solution
        idx_npv = np.argmin(F[:,1])  # Keep as argmin since F[:,1] is -NPV
        best_npv = -F[idx_npv,1]     # Negate to get actual NPV value
        batt_npv = X[idx_npv]
        
        # Store data for later analysis
        self.data['gen'].append(algorithm.n_gen)
        self.data['best_irr'].append(best_irr)
        self.data['best_npv'].append(best_npv)
        self.data['batt_irr'].append(batt_irr)
        self.data['batt_npv'].append(batt_npv)
        self.data['time_elapsed'].append(elapsed)
        
        # Calculate remaining time
        avg_time_per_eval = elapsed / self.evaluations
        remaining_evals = (algorithm.termination.n_max_gen - algorithm.n_gen) * len(X)
        eta = avg_time_per_eval * remaining_evals
        
        # Progress and ETA
        progress = algorithm.n_gen / algorithm.termination.n_max_gen * 100
        progress_bar = "█" * int(progress/5) + " " * (20 - int(progress/5))
        
        # Format NPV as negative or positive as appropriate
        npv_str = f"${best_npv:,.0f}" if best_npv >= 0 else f"-${-best_npv:,.0f}"
        
        print(
            f"Gen {algorithm.n_gen:>2d}/{algorithm.termination.n_max_gen} "
            f"[{progress_bar}] {progress:.1f}% "
            f"[{timedelta(seconds=int(elapsed))}<{timedelta(seconds=int(eta))}] "
            f"Best IRR → {best_irr*100:5.2f}% @ {batt_irr:5.1f} kWh | "
            f"Best NPV → {npv_str} @ {batt_npv:5.1f} kWh"
        )
print("Callback with progress tracking defined successfully")

Callback with progress tracking defined successfully


In [8]:
# Cell 8: Run NSGA‑II optimization
print("Running NSGA‑II optimization…")
total_start_time = time.time()
callback = BestSolutionCallback()

# Population size and number of generations
pop_size = 40
n_generations = 50
print(f"Population size: {pop_size}, Generations: {n_generations}")
print(f"Total evaluations: {pop_size * n_generations}")

res = minimize(
    problem,
    NSGA2(pop_size=pop_size),
    get_termination("n_gen", n_generations),
    seed=42,
    verbose=False,
    callback=callback
)

# Extract & save Pareto front
battery_sizes = res.X.flatten()
pareto_F = res.F
irr_vals = -pareto_F[:,0]  # Convert from -IRR to IRR (higher is better)
npv_vals = -pareto_F[:,1]  # Convert from -NPV to NPV (higher is better)

df = pd.DataFrame({
    'battery_kwh': battery_sizes,
    'IRR': irr_vals,
    'NPV': npv_vals       # Changed from NPC to NPV
})

# Total runtime
total_elapsed = time.time() - total_start_time
print(f"\nOptimization complete! Total runtime: {timedelta(seconds=int(total_elapsed))}")
print(f"Average time per evaluation: {total_elapsed/(pop_size * n_generations):.3f} seconds")

# Save the Pareto front to CSV
(out_dir / 'pareto_solutions.csv').write_text(df.to_csv(index=False))
print(f"✅ Pareto front saved to {out_dir/'pareto_solutions.csv'}")

# Display best solutions
best_irr_idx = df['IRR'].idxmax()
best_npv_idx = df['NPV'].idxmax()
print("\nBest solutions found:")
print(f"Best IRR: {df['IRR'].max()*100:.2f}% with {df.loc[best_irr_idx, 'battery_kwh']:.1f} kWh battery")
print(f"Best NPV: ${df['NPV'].max():,.2f} with {df.loc[best_npv_idx, 'battery_kwh']:.1f} kWh battery")

# Create a scatter plot of the Pareto front
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.scatter(df['IRR']*100, df['NPV'], c=df['battery_kwh'], cmap='viridis', s=50, alpha=0.7)
plt.colorbar(label='Battery Size (kWh)')
plt.xlabel('IRR (%)')
plt.ylabel('NPV ($)')
plt.title('Pareto Front: IRR vs NPV')
plt.grid(True, linestyle='--', alpha=0.7)

# Annotate best points
plt.annotate(f"Best IRR: {df['IRR'].max()*100:.1f}%\nBatt: {df.loc[best_irr_idx, 'battery_kwh']:.1f}kWh",
             xy=(df.loc[best_irr_idx, 'IRR']*100, df.loc[best_irr_idx, 'NPV']),
             xytext=(10, -30), textcoords='offset points',
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=.2'))

plt.annotate(f"Best NPV: ${df['NPV'].max():,.0f}\nBatt: {df.loc[best_npv_idx, 'battery_kwh']:.1f}kWh",
             xy=(df.loc[best_npv_idx, 'IRR']*100, df.loc[best_npv_idx, 'NPV']),
             xytext=(-70, 30), textcoords='offset points',
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=.2'))

plt.tight_layout()
plt.savefig(out_dir / 'pareto_front_irr_npv.png', dpi=300)
plt.close()

print(f"✅ Pareto front visualization saved to {out_dir/'pareto_front_irr_npv.png'}")

# Save convergence data
convergence_df = pd.DataFrame({
    'Generation': callback.data['gen'],
    'Best_IRR': [irr * 100 for irr in callback.data['best_irr']],
    'Best_NPV': callback.data['best_npv'],
    'Battery_IRR': callback.data['batt_irr'],
    'Battery_NPV': callback.data['batt_npv'],
    'Time_Elapsed': callback.data['time_elapsed']
})
convergence_df.to_csv(out_dir / 'convergence_data.csv', index=False)
print(f"✅ Convergence data saved to {out_dir/'convergence_data.csv'}")

Running NSGA‑II optimization…
Population size: 40, Generations: 50
Total evaluations: 2000
Gen  1/50 [                    ] 2.0% [0:00:51<0:42:15] Best IRR →   nan% @  14.6 kWh | Best NPV → -$729 @   1.0 kWh
Gen  2/50 [                    ] 4.0% [0:01:44<0:41:41] Best IRR →   nan% @  14.1 kWh | Best NPV → -$729 @   1.0 kWh
Gen  3/50 [█                   ] 6.0% [0:02:38<0:41:20] Best IRR →   nan% @   0.1 kWh | Best NPV → -$159 @   0.1 kWh
Gen  4/50 [█                   ] 8.0% [0:03:30<0:40:16] Best IRR →   nan% @   0.0 kWh | Best NPV → -$99 @   0.0 kWh
Gen  5/50 [██                  ] 10.0% [0:04:24<0:39:39] Best IRR →   nan% @   0.0 kWh | Best NPV → -$99 @   0.0 kWh
Gen  6/50 [██                  ] 12.0% [0:05:18<0:38:57] Best IRR →   nan% @   0.0 kWh | Best NPV → -$97 @   0.0 kWh
Gen  7/50 [██                  ] 14.0% [0:06:10<0:37:54] Best IRR →   nan% @   0.0 kWh | Best NPV → -$95 @   0.0 kWh
Gen  8/50 [███                 ] 16.0% [0:07:01<0:36:52] Best IRR →   nan% @   0.0 kWh | Be

/var/folders/g6/9fg8n5f101lc1m85rymtg6700000gn/T/ipykernel_62054/2523742622.py:43: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  best_irr_idx = df['IRR'].idxmax()


KeyError: nan

In [ ]:
# Cell 9: Save Full 30-Year Time Series and Data Dictionary
# Run the simulation again for the best solution to get detailed results
try:
    # Try to find best IRR solution
    best_irr_idx = df['IRR'].idxmax()
    
    # Check if the index is valid
    if pd.isna(best_irr_idx):
        # If all IRR values are NaN, use the solution with the best NPV
        print("No valid IRR found. Using solution with best NPV instead.")
        best_npv_idx = df['NPV'].idxmax()  # Since NPV is now positive (we negated NPC)
        best_battery_kwh = df.loc[best_npv_idx, 'battery_kwh']
    else:
        # If a valid IRR exists, use that solution
        best_battery_kwh = df.loc[best_irr_idx, 'battery_kwh']
except Exception as e:
    # Fallback if errors occur: find solution with minimum NPC/maximum NPV
    print(f"Error finding optimal solution based on IRR: {str(e)}")
    print("Using solution with best NPV instead.")
    best_npv_idx = df['NPV'].idxmax()
    best_battery_kwh = df.loc[best_npv_idx, 'battery_kwh']

print(f"\nSimulating best solution for detailed 30-year time series...")
print(f"Selected battery size: {best_battery_kwh:.2f} kWh")
start_time = time.time()

# If best battery size is 0, use a small non-zero value for demonstration
if best_battery_kwh == 0:
    print("Best battery size is 0 kWh. Using 1 kWh for demonstration purposes.")
    analysis_battery_kwh = 1.0
else:
    analysis_battery_kwh = best_battery_kwh

# Calculate battery cost using formula
if analysis_battery_kwh <= 0:
    battery_cost_per_kwh = 0.0  # No cost for 0 kWh battery
else:
    battery_cost_per_kwh = 977.54 * np.exp(-0.004 * analysis_battery_kwh)
    battery_cost_per_kwh = max(600.0, battery_cost_per_kwh)  # Minimum $600/kWh
print(f"Battery cost: ${battery_cost_per_kwh:.2f} per kWh")

# Run simulation with selected battery size and updated parameters
best_disp, best_tots = simulate_battery_dispatch(
    pv_profile['simulated_kwh'],
    demand_profile,
    battery_kwh=analysis_battery_kwh,
    battery_kw=analysis_battery_kwh * 0.5,  # 0.5C rate
    roundtrip_eff=0.9,                      # 90% round trip efficiency
    min_soc_pct=0.05,                       # 95% depth of discharge
    annual_deg_rate=0.01,                   # 1% annual degradation
    grid_emission_rate=0.81                 # 0.81 kg CO2e/kWh
)
elapsed = time.time() - start_time
print(f"Detailed simulation completed in {elapsed:.1f} seconds ({timedelta(seconds=int(elapsed))})")

# Report battery cycles and degradation
print(f"Battery cycles over 30 years: {best_tots['battery_cycles']:.1f}")
print(f"Final battery degradation: {best_tots['final_degradation_pct']:.1f}%")

# Calculate financial metrics for the best solution in detail
best_fin = compute_financials(
    best_tots,
    battery_kwh=analysis_battery_kwh,
    pv_kw=total_pv_capacity,
    pv_cost_per_kw=0.0,                    # Existing PV system
    pv_installation_cost=0.0,              # Existing PV system
    battery_cost_per_kwh=battery_cost_per_kwh,  # Use calculated cost
    battery_installation_cost_per_kwh=174.0,
    battery_power_ratio=0.5,               # Power rating as fraction of capacity
    pv_maintenance_per_kw_day=0.13,        # $0.13 per kW per day for PV
    battery_maintenance_per_kw_day=0.12,   # $0.12 per kW per day for battery
    discount_rate=0.07,
    baseline_import_cost=BASE_IMPORT_COST
)

# Save full 30-year time series to CSV
print(f"Saving full 30-year time series data... ({len(best_disp)} rows)")
best_disp.to_csv(out_dir / "best_irr_30yr_timeseries.csv")
print(f"Saved 30-year time series to {out_dir / 'best_irr_30yr_timeseries.csv'}")

# Create a data dictionary with variable definitions and store as CSV
data_dict = pd.DataFrame([
    {"Variable": "pv_gen", "Description": "PV generation in kWh for each half-hour", "Units": "kWh"},
    {"Variable": "demand", "Description": "Electricity demand in kWh for each half-hour", "Units": "kWh"},
    {"Variable": "pv_used", "Description": "PV generation used directly to satisfy demand", "Units": "kWh"},
    {"Variable": "battery_charge", "Description": "Energy used to charge the battery", "Units": "kWh"},
    {"Variable": "battery_discharge", "Description": "Energy discharged from the battery", "Units": "kWh"},
    {"Variable": "battery_soc", "Description": "Battery state of charge", "Units": "kWh"},
    {"Variable": "pv_export", "Description": "Excess PV generation exported to the grid", "Units": "kWh"},
    {"Variable": "grid_import_peak", "Description": "Electricity imported during peak hours", "Units": "kWh"},
    {"Variable": "grid_import_offpeak", "Description": "Electricity imported during off-peak hours", "Units": "kWh"}
])
data_dict.to_csv(out_dir / "data_dictionary.csv", index=False)
print(f"Saved data dictionary to {out_dir / 'data_dictionary.csv'}")

# Create comprehensive summary
summary_df = pd.DataFrame({
    "Parameter": ["Optimal Battery Size", "PV Size", "Battery Cost", "IRR", "NPV", "Renewable Fraction", "Battery Cycles", "Final Degradation"],
    "Value": [
        best_battery_kwh,  # The actual optimal size (may be 0)
        total_pv_capacity, 
        battery_cost_per_kwh,
        best_fin["irr"] if best_fin["irr"] is not None else "Not calculable",
        best_fin["net_cost"], 
        best_tots["renewable_fraction"],
        best_tots["battery_cycles"],
        best_tots["final_degradation_pct"]
    ],
    "Units": ["kWh", "kW", "$/kWh", "fraction", "$", "fraction", "cycles", "%"],
    "Description": [
        "Optimal battery size (0 = no battery is optimal)", 
        "Fixed PV system capacity", 
        f"Battery cost using formula: 977.54*e^(-0.004*x) with $600 minimum",
        "Internal Rate of Return (may be undefined)", 
        "Net Present Cost", 
        "Fraction of demand met by renewable sources",
        "Total battery charge/discharge cycles over 30 years",
        "Final battery capacity degradation percentage"
    ]
})
summary_df.to_csv(out_dir / "optimization_summary.csv", index=False)
print(f"Saved optimization summary to {out_dir / 'optimization_summary.csv'}")

# Note about the results
if best_battery_kwh == 0:
    print("\nIMPORTANT NOTE: The optimization found that a 0 kWh battery (no battery) is optimal.")
    print("This means batteries are not economically justified with current parameters.")
    print("Consider adjusting electricity rates, battery costs, or maintenance costs to create")
    print("scenarios where batteries provide positive economic value.")
    print("\nFor demonstration purposes, the analysis was conducted with a 1 kWh battery.")

print("\nAnalysis complete!")

In [25]:
# Cell 10: Create daily profile plots and export final results
print("Creating daily profile plots...")
# Summer day
import matplotlib.pyplot as plt
summer_date = pd.Timestamp(f"{demand_profile.index[0].year}-01-15")
summer_mask = (best_disp.index.date == summer_date.date())
summer_data = best_disp[summer_mask]

plt.figure(figsize=(12, 6))
plt.plot(summer_data.index.strftime('%H:%M'), summer_data['pv_gen'], 'orange', label='PV Generation')
plt.plot(summer_data.index.strftime('%H:%M'), summer_data['demand'], 'blue', label='Demand')
plt.plot(summer_data.index.strftime('%H:%M'), summer_data['battery_discharge'], 'green', label='Battery Discharge')
plt.plot(summer_data.index.strftime('%H:%M'), summer_data['grid_import_peak'] + summer_data['grid_import_offpeak'], 'red', label='Grid Import')
plt.fill_between(summer_data.index.strftime('%H:%M'), summer_data['battery_soc'], alpha=0.3, color='green', label='Battery SOC')
plt.xticks(rotation=90)
plt.title(f'Summer Day Profile: {summer_date.strftime("%B %d")}')
plt.xlabel('Time of Day')
plt.ylabel('Energy (kWh)')
plt.legend()
plt.tight_layout()
plt.savefig(out_dir / "summer_day_profile.png", dpi=300)
plt.close()

# Winter day
winter_date = pd.Timestamp(f"{demand_profile.index[0].year}-07-15")
winter_mask = (best_disp.index.date == winter_date.date())
winter_data = best_disp[winter_mask]

plt.figure(figsize=(12, 6))
plt.plot(winter_data.index.strftime('%H:%M'), winter_data['pv_gen'], 'orange', label='PV Generation')
plt.plot(winter_data.index.strftime('%H:%M'), winter_data['demand'], 'blue', label='Demand')
plt.plot(winter_data.index.strftime('%H:%M'), winter_data['battery_discharge'], 'green', label='Battery Discharge')
plt.plot(winter_data.index.strftime('%H:%M'), winter_data['grid_import_peak'] + winter_data['grid_import_offpeak'], 'red', label='Grid Import')
plt.fill_between(winter_data.index.strftime('%H:%M'), winter_data['battery_soc'], alpha=0.3, color='green', label='Battery SOC')
plt.xticks(rotation=90)
plt.title(f'Winter Day Profile: {winter_date.strftime("%B %d")}')
plt.xlabel('Time of Day')
plt.ylabel('Energy (kWh)')
plt.legend()
plt.tight_layout()
plt.savefig(out_dir / "winter_day_profile.png", dpi=300)
plt.close()

print("Profile plots created and saved to:", out_dir)

# Export all remaining data
# 1) Roof configuration
pd.DataFrame(roof_params).to_csv(out_dir / 'roof_config.csv', index=False)

# 2) Weather files list
pd.DataFrame({'weather_file': weather_files}).to_csv(out_dir / 'weather_files.csv', index=False)

# 3) Best solution details as text file
with open(out_dir / "best_solution_details.txt", "w") as f:
    f.write(f"Best Solution Details:\n")
    f.write(f"PV Size: {total_pv_capacity:.1f} kW\n")
    f.write(f"Battery Size: {best_battery_kwh:.1f} kWh\n")
    
    # Calculate battery cost for text file
    if analysis_battery_kwh <= 0:
        battery_cost = 0.0
    else:
        battery_cost = 977.54 * np.exp(-0.004 * analysis_battery_kwh)
        battery_cost = max(600.0, battery_cost)
    f.write(f"Battery Cost: ${battery_cost:.2f} per kWh\n")
    
    f.write(f"IRR: {best_fin['irr']:.2%}\n") if best_fin['irr'] is not None else f.write(f"IRR: Not calculable\n")
    f.write(f"Net Present Cost: ${best_fin['net_cost']:.2f}\n")
    f.write(f"\nSystem Configuration:\n")
    f.write(f"PV Tilt: {roof_params[0]['tilt']}°\n")
    f.write(f"PV Azimuth: {roof_params[0]['azimuth']}°\n")
    f.write(f"PV Shading: {roof_params[0]['shading']}%\n")
    f.write(f"\nSimulation Parameters:\n")
    f.write(f"PV Cost: $0 (existing system)\n")
    f.write(f"Battery Cost: Formula: 977.54*e^(-0.004*x) with $600 minimum\n")
    f.write(f"Battery Installation Cost: $174/kWh\n")
    f.write(f"PV Maintenance: $0.13/kW/day\n")
    f.write(f"Battery Maintenance: $0.12/kW/day\n")
    f.write(f"Discount Rate: 7%\n")
    f.write(f"Battery Degradation: 1%/year\n")
    f.write(f"Battery Depth of Discharge: 95%\n")
    f.write(f"Battery Round Trip Efficiency: 90%\n")
    f.write(f"Battery C-Rate: 0.5C\n")
    f.write(f"Grid Emission Rate: 0.81 kg CO2e/kWh\n")
    f.write(f"\nEnergy Summary:\n")
    f.write(f"Total Demand: {best_tots['total_demand']:.2f} kWh\n")
    f.write(f"Total PV Used Directly: {best_tots['total_pv_used']:.2f} kWh\n")
    f.write(f"Total Battery Discharge: {best_tots['total_battery_discharge']:.2f} kWh\n")
    f.write(f"Total Grid Import (Peak): {best_tots['total_grid_import_peak']:.2f} kWh\n")
    f.write(f"Total Grid Import (Off-Peak): {best_tots['total_grid_import_offpeak']:.2f} kWh\n")
    f.write(f"Total PV Export: {best_tots['total_pv_export']:.2f} kWh\n")
    f.write(f"Renewable Fraction: {best_tots['renewable_fraction']:.2%}\n")
    f.write(f"Self-Consumption Rate: {best_tots['self_consumption_rate']:.2%}\n")
    f.write(f"Battery Cycles: {best_tots['battery_cycles']:.1f}\n")
    f.write(f"Final Battery Degradation: {best_tots['final_degradation_pct']:.1f}%\n")
# 4) Financial summary
pd.DataFrame.from_dict(best_fin, orient='index', columns=['value']).to_csv(out_dir / 'financial_summary.csv')

# 5) Dispatch totals  
pd.DataFrame.from_dict(best_tots, orient='index', columns=['value']).to_csv(out_dir / 'dispatch_totals.csv')

# 6) Convergence data
convergence_df = pd.DataFrame({
    'Generation': callback.data['gen'],
    'Best_IRR': [irr * 100 for irr in callback.data['best_irr']],
    'Best_NPV': callback.data['best_npv'],
    'Battery_IRR': callback.data['batt_irr'],
    'Battery_NPV': callback.data['batt_npv'],
    'Time_Elapsed': callback.data['time_elapsed']
})
convergence_df.to_csv(out_dir / 'convergence_data.csv', index=False)

print(f"\n✅ All outputs successfully saved to {out_dir}")
print("NSGA-II optimization complete!")

Creating daily profile plots...
Profile plots created and saved to: /Users/petertunali/Documents/GitHub/Battery_Optimisation/5_nsga_outputs/003


KeyError: 'best_npc'

In [ ]:
# Cell 11: Calculate and display electricity cost savings with battery
print("Calculating electricity cost comparison...")
try:
    # Get the best IRR solution
    best_irr_idx = df['IRR'].idxmax()
    best_battery = df.loc[best_irr_idx, 'battery_kwh']
    
    # Calculate for first year only (17520 points)
    # 1) Annual cost without PV or battery (baseline)
    annual_demand = demand_profile[:17520].sum()  
    annual_peak_demand = 0
    annual_offpeak_demand = 0
    
    # Calculate peak/offpeak split based on TOD tariff
    print("Calculating peak/off-peak demand split...")
    for idx, val in demand_profile[:17520].items():
        h = idx.hour
        m = idx.month
        # Summer (Oct-Mar): 2pm-8pm, Winter (Apr-Sep): 3pm-9pm
        if (m < 4 or m > 9):  # Summer
            is_peak = (h >= 14 and h < 20)
        else:  # Winter
            is_peak = (h >= 15 and h < 21)
        
        if is_peak:
            annual_peak_demand += val
        else:
            annual_offpeak_demand += val
    
    baseline_cost = annual_peak_demand * 0.39710 + annual_offpeak_demand * 0.13530
    
    # 2) Annual cost with PV but no battery
    print("Calculating costs with PV only...")
    disp_pv_only, tots_pv_only = simulate_battery_dispatch(
        pv_gen=pv_profile['simulated_kwh'][:17520],
        demand=demand_profile[:17520],
        battery_kwh=0.0
    )
    
    pv_only_cost = (tots_pv_only['total_grid_import_peak'] * 0.39710 + 
                    tots_pv_only['total_grid_import_offpeak'] * 0.13530 - 
                    tots_pv_only['total_pv_export'] * 0.033)
    
    # 3) Annual cost with PV and battery
    print(f"Calculating costs with PV and {best_battery:.1f} kWh battery...")
    disp_pv_batt, tots_pv_batt = simulate_battery_dispatch(
        pv_gen=pv_profile['simulated_kwh'][:17520],
        demand=demand_profile[:17520],
        battery_kwh=best_battery,
        battery_kw=best_battery * 0.5,
        roundtrip_eff=0.9,
        min_soc_pct=0.2,
        annual_deg_rate=0.01
    )
    
    pv_batt_cost = (tots_pv_batt['total_grid_import_peak'] * 0.39710 + 
                    tots_pv_batt['total_grid_import_offpeak'] * 0.13530 - 
                    tots_pv_batt['total_pv_export'] * 0.033)
    
    # Calculate savings
    savings_pv_only = baseline_cost - pv_only_cost
    savings_pv_batt = baseline_cost - pv_batt_cost
    additional_savings = pv_only_cost - pv_batt_cost
    
    # Save results to file
    savings_df = pd.DataFrame({
        'Scenario': ['Baseline (no PV/battery)', 'PV Only', f'PV + {best_battery:.1f}kWh Battery'],
        'Annual Cost ($)': [baseline_cost, pv_only_cost, pv_batt_cost],
        'Savings vs Baseline ($)': [0, savings_pv_only, savings_pv_batt],
        'Electricity Cost Reduction (%)': [0, savings_pv_only/baseline_cost*100, savings_pv_batt/baseline_cost*100]
    })
    savings_df.to_csv(out_dir / 'annual_electricity_savings.csv', index=False)
    
    # Print results
    print("\n=== Annual Electricity Cost Comparison ===")
    print(f"Baseline (no PV, no battery): ${baseline_cost:.2f}")
    print(f"With PV only:                 ${pv_only_cost:.2f}")
    print(f"With PV + {best_battery:.1f}kWh battery:    ${pv_batt_cost:.2f}")
    print("\n=== Annual Savings ===")
    print(f"Savings from PV only:            ${savings_pv_only:.2f}")
    print(f"Savings from PV + battery:       ${savings_pv_batt:.2f}")
    print(f"Additional savings from battery:  ${additional_savings:.2f}")
    print(f"This represents a {additional_savings/pv_only_cost*100:.1f}% reduction in electricity costs compared to PV-only")
    
except Exception as e:
    print(f"Error calculating electricity costs: {e}")